In [36]:
import sqlite3
conn = sqlite3.connect("AQI.db")
with conn:
    sql = '''
    CREATE TABLE IF NOT EXISTS records (
        id INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT,
        sitename TEXT NOT NULL,
        county TEXT,
        aqi INTEGER,
        status TEXT,
        pm25 NUMERIC,
        date TEXT,
        lat NUMERIC,
        lon NUMERIC,
        UNIQUE(sitename,date)
    );
    '''
    cursor = conn.cursor()
    cursor.execute(sql)

In [37]:
import requests

url = 'https://data.moenv.gov.tw/api/v2/aqx_p_488?api_key=e8dd42e6-9b8b-43f8-991e-b3dee723a52d&limit=1000&sort=datacreationdate%20desc&format=JSON'
try:
    response = requests.get(url)
    response.raise_for_status()
    data = response.json()
except Exception as e:
    print(e)
else:
    sitenames = set()
    for items in data['records']:
        sitenames.add(items['sitename'])

    sitenames = list(sitenames)
print(sitenames)

['埔里', '新北(樹林)', '屏東', '新竹', '線西', '基隆', '花蓮', '竹東', '中壢', '臺南（麻豆）', '高雄（湖內）', '林口', '前金', '朴子', '嘉義', '豐原', '新店', '新莊', '古亭', '中山', '崙背', '潮州', '松山', '宜蘭', '臺西', '桃園', '湖口', '大城', '新港', '恆春', '龍潭', '永和', '屏東(枋山)', '土城', '美濃', '金門', '苗栗', '橋頭', '觀音', '臺東', '三重', '三義', '彰化', '鳳山', '關山', '菜寮', '屏東（琉球）', '冬山', '忠明', '楠梓', '二林', '馬公', '萬華', '麥寮', '士林', '前鎮', '小港', '林園', '南投', '彰化（員林）', '大寮', '馬祖', '宜蘭（頭城）', '富貴角', '淡水', '板橋', '竹山', '頭份', '大同', '西屯', '左營', '汐止', '安南', '萬里', '平鎮', '陽明', '大里', '大園', '斗六', '復興', '善化', '臺南', '沙鹿', '仁武', '新營']


In [38]:

import datasource
datasource.get_selected_data(sitename='屏東(枋山)')

AttributeError: module 'datasource' has no attribute 'get_selected_data'

In [16]:

import sqlite3
conn = sqlite3.connect("AQI.db")
with conn:
    sql = '''INSERT OR IGNORE INTO records(sitename,county,aqi,status,pm25,date,lat,lon)
    values ('屏東(枋山)','屏東縣', 34, '良好',18,'2024-11-04 10:00', 22.260899, 120.651472);
    '''
    cursor = conn.cursor()
    cursor.execute(sql)

In [33]:
import sqlite3
conn = sqlite3.connect("AQI.db")
url = 'https://data.moenv.gov.tw/api/v2/aqx_p_488?api_key=e8dd42e6-9b8b-43f8-991e-b3dee723a52d&limit=1000&sort=datacreationdate%20desc&format=JSON'
try:
    response = requests.get(url)
    response.raise_for_status()
    data = response.json()
except Exception as e:
    print(e)
else:
    sitenames = set()
    with conn:
        cursor = conn.cursor()
        for items in data['records']:
            sitename = items['sitename']
            county = items['county']
            aqi = int(items['aqi']) if items['aqi'] != '' else 0
            status = items['status']
            pm25 = float(items['pm2.5']) if items['pm2.5'] != '' else 0.0
            date = items['datacreationdate']
            lon = float(items['longitude']) if items['longitude'] != '' else 0.0
            lat = float(items['latitude']) if items['latitude'] != '' else 0.0
            sql = '''INSERT OR IGNORE INTO records(sitename,county,aqi,status,pm25,date,lat,lon)
                    values (?,?, ?, ?,?,?,?,?);
            '''
            cursor.execute(sql,(sitename, county, aqi, status,pm25,date,lat,lon))

In [32]:
conn = sqlite3.connect("AQI.db")
with conn:
    cursor = conn.cursor()
    sql = '''
    SELECT DISTINCT sitename
    FROM records
    '''
    cursor.execute(sql)
    sitenames = []

    for items in cursor.fetchall():
        sitenames.append(items[0])
    
    print(sitenames)

['三義', '三重', '中壢', '中山', '二林', '仁武', '冬山', '前金', '前鎮', '南投', '古亭', '善化', '嘉義', '土城', '埔里', '基隆', '士林', '大同', '大園', '大城', '大寮', '大里', '安南', '宜蘭', '宜蘭（頭城）', '富貴角', '小港', '屏東', '屏東(枋山)', '屏東（琉球）', '崙背', '左營', '平鎮', '彰化', '彰化（員林）', '復興', '忠明', '恆春', '斗六', '新北(樹林)', '新店', '新港', '新營', '新竹', '新莊', '朴子', '松山', '板橋', '林口', '林園', '桃園', '楠梓', '橋頭', '永和', '汐止', '沙鹿', '淡水', '湖口', '潮州', '竹山', '竹東', '線西', '美濃', '臺南', '臺南（麻豆）', '臺東', '臺西', '花蓮', '苗栗', '菜寮', '萬華', '萬里', '西屯', '觀音', '豐原', '金門', '關山', '陽明', '頭份', '馬公', '馬祖', '高雄（湖內）', '鳳山', '麥寮', '龍潭']


In [39]:
conn = sqlite3.connect("AQI.db")
with conn:
    cursor = conn.cursor()        
    sql = '''
    SELECT date,county,aqi,pm25,status,lat,lon
    FROM records
    WHERE sitename=?
    ORDER BY date DESC;
    '''
    cursor.execute(sql,('富貴角',))
    sitename_list = [list(item) for item in cursor.fetchall()]
    print(sitename_list)

[['2024-11-04 14:00', '新北市', 0, 0, '', 25.29681695, 121.53656894], ['2024-11-04 13:00', '新北市', 47, 8, '良好', 25.29681695, 121.53656894], ['2024-11-04 12:00', '新北市', 48, 0, '良好', 25.29681695, 121.53656894], ['2024-11-04 11:00', '新北市', 48, 0, '良好', 25.29681695, 121.53656894], ['2024-11-04 10:00', '新北市', 48, 13, '良好', 25.29681695, 121.53656894], ['2024-11-04 09:00', '新北市', 49, 14, '良好', 25.29681695, 121.53656894], ['2024-11-04 08:00', '新北市', 49, 13, '良好', 25.29681695, 121.53656894], ['2024-11-04 07:00', '新北市', 50, 10, '良好', 25.29681695, 121.53656894], ['2024-11-04 06:00', '新北市', 51, 7, '普通', 25.29681695, 121.53656894], ['2024-11-04 05:00', '新北市', 55, 18, '普通', 25.29681695, 121.53656894]]
